# 04. Semantic Kernel

In this lab, we will introduce **Semantic Kernel**. Like **Langchain**, it provides a framework for working with AI models and supports applications written in Python. Semantic Kernel also adds support for .NET and Java applications.

As with the other labs, we'll start by reading in values from the `.env` file.

___
**NOTE**: This lab uses **.NET** so be sure to select .NET and not Python when starting to use this notebook.
___

In [ ]:
#r "nuget: dotenv.net, 3.1.2"

using dotenv.net;

// Read values from .env file
var envVars = DotEnv.Fluent()
    .WithoutExceptions()
    .WithEnvFiles("../../../.env")
    .WithTrimValues()
    .WithDefaultEncoding()
    .WithOverwriteExistingVars()
    .WithoutProbeForEnv()
    .Read();

// Load values into variables and strip quotes
var model = envVars["AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"].Replace("\"", "");
var azureEndpoint = envVars["AZURE_OPENAI_ENDPOINT"].Replace("\"", "");
var apiKey = envVars["AZURE_OPENAI_API_KEY"].Replace("\"", "");


To start working with Semantic Kernel, we'll need to load its nuget package.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.10.0"

Semantic Kernel works by creating an instance of the Kernel and then adding in various plugins to perform different functions. Those plugins or functions can then be called individually or chained together to perform more complex tasks.

We use the standard .NET `builder` pattern to initialise the kernel. Notice that we pass in the details of the completion model that we're going to use, the Azure OpenAI API endpoint URL and the API key.

In [ ]:
using Microsoft.SemanticKernel;

var builder = Kernel.CreateBuilder();
builder.Services.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
var kernel = builder.Build();

## Send a prompt to Azure OpenAI using Semantic Kernel

Now that we've established a connection to the Azure OpenAI API, we can go ahead and send a prompt to the LLM.

In [ ]:
Console.WriteLine(await kernel.InvokePromptAsync("What things could I make with a Raspberry Pi?"));

Let's take that simple prompt forward and create a function with a prompt template to perform a simple request to Azure OpenAI. The template allows us to define a prompt and add placeholders for values that we will provide later. These values could come from user input, or another function, for example.

In [ ]:
var whatCanIMakeFunction = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig()
    {
        Template = @"What interesting things can I make with a {{$item}}?",
        InputVariables = [
            new() { Name = "item", Description = "An item to make something with.", IsRequired=true }
        ]
    });


`{{$item}}` represents the value we will provide later and will be replaced when we make the call to Azure OpenAI.

Next, we'll define a value for `item` and then call the function, passing in the `item` as we do so.

In [ ]:
string item = "raspberry pi";

var response = await kernel.InvokeAsync(whatCanIMakeFunction, new () { { "item", item }});

Console.WriteLine(response);

The result is a list of ideas for things we could make with a Raspberry Pi. 

We could then use Semantic Kernel to take that list of ideas and summarize it. First, we'll create a new prompt that will generate a summary of some text.

In [ ]:
var thingsToMakeSummary = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig()
    {
        Template = @"Summarize the following text: {{$thingsToMake}}?",
        InputVariables = [
            new() { Name = "thingsToMake", Description = "A list of things you could make.", IsRequired=true }
        ]
    });

Next, we can call the kernel again, this time passing in the list of ideas, the `response` from the previous call, as the value for `thingsToMake`.

In [ ]:
var summary = await kernel.InvokeAsync(thingsToMakeSummary, new () { { "thingsToMake", response }});

Console.WriteLine(summary);

## Summary

Semantic Kernel is another example of an AI orchestrator. Like other orchestrators, it can be used to simplify the process of creating complex AI applications.

## Next Section

📣 [AI Orchestration](../../03-orchestration/README.md)